In [2]:
"""
@author: Pablo
"""
import json
import matplotlib.pyplot as plt
import pandas
import ffmpeg
import skimage
import numpy as np
import os
import cv2
path = "D:\\Uni work\\Engineering Mathematics Work\\MDM3\Mojo\\mojo_sperm_tracking_data_bristol\\tp49\\"
os.chdir(path)

In [3]:
# open first video and save the json file as data
with open(r"D:\Uni work\Engineering Mathematics Work\MDM3\Mojo\mojo_sperm_tracking_data_bristol\tp49\cover0_0_YOLO_NO_TRACKING_output\centroids_with_meta.json", "r") as read_file:
    data = json.load(read_file)

In [4]:
# print data with indentation for readability 
print(json.dumps(data, sort_keys=True, indent =2))

{
  "centroids": [
    [
      {
        "bbox": [
          321.0,
          205.0,
          31.0,
          32.0
        ],
        "center": [
          337.25,
          221.0
        ],
        "class": 1,
        "id": "0000",
        "interpolated": false
      },
      {
        "bbox": [
          538.0,
          128.0,
          34.0,
          33.0
        ],
        "center": [
          555.75,
          145.25
        ],
        "class": 1,
        "id": "0001",
        "interpolated": false
      },
      {
        "bbox": [
          701.0,
          422.0,
          32.0,
          32.0
        ],
        "center": [
          717.75,
          438.25
        ],
        "class": 1,
        "id": "0002",
        "interpolated": false
      },
      {
        "bbox": [
          67.0,
          507.0,
          34.0,
          34.0
        ],
        "center": [
          84.5,
          524.0
        ],
        "class": 1,
        "id": "0003",
        "interpolated":

In [5]:

print(data['centroids'])

[[{'bbox': [321.0, 205.0, 31.0, 32.0], 'center': [337.25, 221.0], 'class': 1, 'interpolated': False, 'id': '0000'}, {'bbox': [538.0, 128.0, 34.0, 33.0], 'center': [555.75, 145.25], 'class': 1, 'interpolated': False, 'id': '0001'}, {'bbox': [701.0, 422.0, 32.0, 32.0], 'center': [717.75, 438.25], 'class': 1, 'interpolated': False, 'id': '0002'}, {'bbox': [67.0, 507.0, 34.0, 34.0], 'center': [84.5, 524.0], 'class': 1, 'interpolated': False, 'id': '0003'}, {'bbox': [204.0, 146.0, 31.0, 31.0], 'center': [219.5, 161.5], 'class': 1, 'interpolated': False, 'id': '0004'}, {'bbox': [829.0, 161.0, 37.0, 35.0], 'center': [847.5, 178.5], 'class': 1, 'interpolated': False, 'id': '0005'}, {'bbox': [889.0, 560.0, 37.0, 37.0], 'center': [908.25, 579.0], 'class': 1, 'interpolated': False, 'id': '0006'}, {'bbox': [844.0, 398.0, 34.0, 35.0], 'center': [861.0, 416.25], 'class': 1, 'interpolated': False, 'id': '0007'}, {'bbox': [937.0, 308.0, 22.0, 31.0], 'center': [948.75, 323.5], 'class': 1, 'interpolated

In [6]:
vid = "cover0_0.avi"
path = "D:\\Uni work\\Engineering Mathematics Work\\MDM3\Mojo\\mojo_sperm_tracking_data_bristol\\tp49\\"


def playvid(path=path, vid=vid):
    vidcap =cv2.VideoCapture(path+vid)
    framerate = vidcap.get(5)
    framecount = vidcap.get(7)
#     print(framerate, framecount)
    while True:
        success, frame = vidcap.read()
        if success:
        
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cv2.imshow('frame',gray)
            cv2.waitKey(1)
        else:
            break

    vidcap.release()
    cv2.destroyAllWindows()

    


def framesplit(path=path, vid=vid, samplerate=10):
    os.chdir(path)
    vidcap =cv2.VideoCapture(path+vid)
    framerate = vidcap.get(5)
    framecount = vidcap.get(7)
    print(framerate, framecount)
    count = 0
    success, frame = vidcap.read()
    print(success)
    while success: 
        cv2.imwrite("frame%d.jpg" % count, frame)     # save frame as JPEG file      
        success,frame = vidcap.read()
        print('Read a new frame: ', success)
        count += 1
    vidcap.release()
#         #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
playvid(path=path, vid="cover1_2.avi")


In [37]:
#loop through all sperms to find their bounding box and center points for plotting
def getbboxes():
    frame_bboxes=[]
    centroids= []
    for li in data['centroids']: 
        bboxes = []
        centers = []
        for obj in li:
            bboxes.append(obj['bbox'])
            centers.append(obj['center'])
#             print(obj['bbox'])
        frame_bboxes.append(bboxes)
        centroids.append(centers)
    return frame_bboxes, centroids



In [38]:

def annotateframe(frame=0):
    frame=frame
    resizefactor = 2
    frame_bboxes, centroids = getbboxes()
    
    cv2.namedWindow('output', cv2.WINDOW_NORMAL)
    img = cv2.imread('frame{}.jpg'.format(str(frame)))
    #imgresized = cv2.resize(img, (960, 600)) 
    # imgray = cv2.cvtColor(imgresized,cv2.COLOR_BGR2GRAY)
   

    for box in frame_bboxes[frame]:

        x_min, y_min, w, h = resizefactor*int(box[0]),resizefactor*int(box[1]),int(box[2]),int(box[3])
        print(x_min, y_min, w, h)
        startpoint = (x_min, y_min+h)
        endpoint =  (x_min+w, y_min)

        # draw a green rectangle to visualize the bounding rect
        img=cv2.rectangle(img, (x_min, y_min,x_min+w, y_min-h), (0, 255, 0), 2)

    for centroid in centroids[frame]:
        x, y = resizefactor* int(centroid[0]), resizefactor* int(centroid[1])
        img = cv2.circle(img, (x,y), radius=3, color=(0, 0, 255), thickness=-1)
    cv2.imshow('output',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
annotateframe(295)


586 386 33 32
434 166 33 33
1300 874 32 31
0 1002 26 33
126 222 34 34
1172 302 35 35
1650 766 33 33
1458 1098 32 32
1754 1114 34 32
1138 482 33 32
1392 924 32 32
1896 562 12 30
